<a href="https://colab.research.google.com/github/simeonnalovski/NLP-Sentiment-Analysis-and-Text-Generation-in-Finance-/blob/main/Text_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 20.5 MB/s eta 0:00:00


In [ ]:
!pip install huggingface-hub

In [ ]:
import torch
import pandas as pd
from google.colab import drive
from transformers import AutoModelForCausalLM, AutoTokenizer
import traceback
from transformers import pipeline
import re

drive.mount('/content/drive')

DATA_PATH = '/content/drive/MyDrive/data/stock_sentiment.csv'


def generate_response(SELECTED_STOCK, total_sentiment, mean_sentiment, sentiment_label):


    content_text = (
    f"You are analyzing stock market sentiment for the company with the ticker symbol {SELECTED_STOCK}, "
    f"based on data from the last 28 days extracted from 100 financial news articles. "
    f"The sentiment analysis gave the following results:\n"
    f"- Mean sentiment score: {mean_sentiment} out of 2 per article\n"
    f"- Total sentiment score: {total_sentiment} out of 200\n"
    f"- Overall sentiment label: {sentiment_label}\n\n"
    f"Based solely on these metrics, provide a concise 2-3 sentence investment recommendation: "
    f"Should an investor Buy, Sell, or Hold this stock? "
    f"Support your recommendation using reasoning derived only from the sentiment data."
    )

    pipe = pipeline("text-generation", model="WiroAI/WiroAI-Finance-Qwen-1.5B",max_new_tokens=500, return_full_text=False)
    messages = [
        {"role": "system", "content": "You are an expert financial assistant in stock investing. Always end each sentence with a period and start each sentence on a new line."},
        {"role": "user", "content": content_text}
    ]

    full_response = ""

    try:
       outputs = pipe(messages)
       full_response= outputs[0]['generated_text']


    except Exception as e:
        print(f"Error in model inference: {e}")
        return

    sentences = re.split(r'(?<=[.!?])\s+', full_response.strip())
    formatted_response = '\n'.join(sentence.strip() for sentence in sentences if sentence.strip())

    print(formatted_response)


try:
    df = pd.read_csv(DATA_PATH)

    SELECTED_STOCK = "META"  # Meta
    stock_data = df[df['stock'] == SELECTED_STOCK]

    if stock_data.empty:
        print(f"No data found for stock: {SELECTED_STOCK}")
    else:
        print(f"Found data for {SELECTED_STOCK}")
        mean_sentiment = stock_data['mean_sentiment'].values[0]
        total_sentiment = stock_data['total_sentiment'].values[0]
        sentiment_label = stock_data['sentiment'].values[0]
        generate_response(SELECTED_STOCK, total_sentiment, mean_sentiment, sentiment_label)

except Exception as e:
    print(f"Error loading data: {e}")
    raise e


Mounted at /content/drive
Found data for META


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/766 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/6.75k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/485 [00:00<?, ?B/s]

Device set to use cpu


Given the overall sentiment label of 'neutral' for the stock with ticker symbol META and the mean sentiment score of 1.1717, the stock does not exhibit strong support or resistance in recent market conditions.
This suggests that the stock may not be an attractive buy at the current price.
Considering the overall sentiment is neutral, the stock could be viewed as sideways or sideways in the short-term market.
Therefore, an investor may choose to hold this stock for the long-term, given its moderate support from the sentiment data.
